# Data Source:
Kaggle (ongoing competition): https://www.kaggle.com/c/prudential-life-insurance-assessment

# Project goal:
Analyze features collected from individuals applying for life insurance at Prudential to predict response of the company.

**Motivation for the problem:**
Once Prudential collects features from its applicants, it takes a long time to come up with a decision. If we can come up with a model that accurately predicts the decisions they come up with, the company can use this model and swiftly determine its decision.

# Notes on dataset:
59381 rows, 127 columns: 126 features (some of which are dummy variables) and 1 outcome column. The outcome column is called **Response**. There are 8 nominal values (1,2,3,4,5,6,7,8) in the Response column.

# Immediate goal: working with a smaller subset of Prudential data 

The point is to be able to go through with a workflow on my laptop. Using the entire datasets chokes Jupyter.
- Plan to use Amazon's EC2 cloud server but for now will work with a subset sampled from the data.
- Data set (59381 rows) is randomly subsetted to a smaller df with 500 rows.

In [1]:
# Imports and magic here
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.manifold import Isomap
from sklearn.cluster import KMeans
from sklearn import cross_validation
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from sklearn.learning_curve import learning_curve
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.cross_validation import train_test_split, StratifiedKFold, ShuffleSplit 
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
# Globals here

In [2]:
# Read input dataset
input_csv = './train.csv'
df_raw = pd.read_csv(input_csv, index_col = 0)

IOError: File ./train.csv does not exist

** Randomly extract 500 rows from df_raw into df_sampled **

In [ ]:
# Sample df_raw to create a smaller, easier to work with df
rows = np.random.choice(df_raw.index.values,500)
df_sampled = df_raw.ix[rows]

In [ ]:
df_sampled.shape

In [ ]:
# Identify predictor variables and response varibales, and create corresponding DFs

def split_response_from_features(df):
    response = 'Response'
    all_predictors = [col for col in df.columns if col not in response]
    df_predictors = df[all_predictors]
    df_response = df[response]
    df_response_with_dummies = pd.get_dummies(df_response,prefix='Response')
    return df_predictors, df_response, df_response_with_dummies

In [ ]:
df_predictors, df_response_with_dummies, df_response = split_response_from_features(df_sampled)

In [ ]:
df_sampled.head()

In [ ]:
df_response.head()

**Preliminary exploratory plots:**
1. Wt versus BMI (Intuitively one would guess there is a positive correlation).
2. Plot the distribution of Response variable to see if there are evenly represented in df_sample. If not we might need to upsample, downsample or generate rows for the underrepresented response using SMOTE.

In [ ]:
# Plot scatterplot of Wt vs BMI
with plt.style.context('fivethirtyeight'):
    plt.scatter(df_sampled.BMI,df_sampled.Wt,s=40)
    # add axis label
    plt.xlabel('BMI')
    plt.ylabel('Wt')
plt.show()    

In [ ]:
# Plot the distribution of Response
sns.distplot(df_sampled.Response)

In [ ]:
sns.countplot(df_sampled.Response, palettle="husl", order=range(1,9))

In [ ]:
response_count = {}
for ii in df_sampled.Response.unique():
    response_count[ii] = len(df_sampled[df_sampled.Response==ii])
    total_num = len(df_sampled[df_sampled.Response==ii])
    print "Total number of Response#{} : {}".format(ii,total_num)
print response_count       

# Notes on the Response column
1. Response 6,8 is roughly 4-5 times higher than the rest of the responses : consider undersampling.
2. Response 3 and 4 are way less than the others: consider oversampling.

In [ ]:
# Describe each feature: Number of uniques, number of nulls, type_of_feature, distribution

def get_dummy_features(df, verbose=False, dummy_threshold = 200):
    features_for_dummification = []
    num_rows = len(df)

    for each_feature in df.columns:
        num_uniques = len(df[each_feature].unique())
        num_nulls = df[each_feature].isnull().sum()
        example = df[each_feature].iloc[0]
        
        # Keep track of dummy features
        if isinstance(example, str) or (num_uniques*dummy_threshold<num_rows and isinstance(example, (int, long))): 
            features_for_dummification.append(each_feature)

        if verbose==True:
            print '{}: Uniques: {}/{}. Nulls: {}. Type: {}'.format(each_feature, num_uniques, 
                                                               num_rows, num_nulls, type_of_feature)
            
            
    return features_for_dummification

In [ ]:
# check if number of dummiable columns in the original df and the sampled df are the same
print len(get_dummy_features(df_raw,verbose=False,dummy_threshold=200))
print len(get_dummy_features(df_sampled,verbose=False,dummy_threshold=200))

In [ ]:
#get_dummy_features(df_predictors)

**The answer is no: 108 for original df_raw and 94 for df_sampled. But that's ok. Just go ahead with the workflow anyway.**

In [ ]:
# Turn chosen features into dummy variables
def create_dummy_features(df, features_for_dummification, verbose=True):
    df_expanded = df

    for each_feature in features_for_dummification:
        if verbose==True:
            print "Expanding variable: {}".format(each_feature)
        df_temp = pd.get_dummies(df[each_feature], prefix=each_feature)
        df_expanded = pd.concat([df_expanded, df_temp], axis = 1, join = 'inner')
        df_expanded.drop(each_feature,inplace=True, axis=1)
        
    return df_expanded

In [ ]:
# # Since training data and test data have different distributions for "dummyizable" features, it makes sense to
# # build out dummy features using a combined distribution
df_oos_sneak_peek = pd.read_csv('./test.csv', index_col = 0)
df_predictors_enhanced = pd.concat([df_predictors, df_oos_sneak_peek], axis = 0, join = 'outer')
df_predictors_enhanced.shape

In [ ]:
# Since Medical_Keyword is already dummied, make sure you exclude it in the dummying process!
features_for_dummification = get_dummy_features(df_predictors)
features_already_dummy = ['Medical_Keyword_{}'.format(num) for num in range(1,49)]
features_for_dummification = sorted(list(set(features_for_dummification) - set(features_already_dummy)))
print features_for_dummification

In [ ]:
# Run the functions from above (df_expanded has dummy variables)
df_sample_expanded_enhanced = create_dummy_features(df_predictors_enhanced,features_for_dummification)
df_expanded = df_sample_expanded_enhanced.drop(df_oos_sneak_peek.index)
df_expanded.shape

** Question about performing correlations: should we test this before making dummies?**

In [ ]:
# # Compute pairwise correlations of all predictors
# df_predictors = df_expanded
# df_predictor_correlations = df_predictors.corr()


In [ ]:
# # Describe pairwise correlations
# _corr_threshold = 0.5

# for pred in all_predictors:
#     df_temp = df_predictor_correlations[pred]
#     max_corr = df_temp[[idx for idx in df_temp.index if pred not in idx]].max()
#     min_corr = df_temp.min()
#     if np.isnan(max_corr)==False:
#         if abs(min_corr)>max_corr:
#             other_pred = df_temp[df_temp==min_corr].index[0]
#             mcorr = min_corr
#         else:
#             other_pred = df_temp[df_temp==max_corr].index[0]
#             mcorr = max_corr
#         if abs(mcorr)>_corr_threshold:
#             print '** ',
#         print 'mcorr({})={:.4f} with {}'.format(pred, mcorr, other_pred)

# Feature Engineering:
- Fill NAs for both the predictord and response with median estimates of the column.
- Consider localized imputing in future.
    

In [ ]:
# Fill NAs with median estimates
df_predictors_selected = df_expanded
df_response_selected = df_response_with_dummies

df_predictors_selected.fillna(df_predictors_selected.median(),inplace=True)
df_response_selected.fillna(df_response_selected.median(),inplace=True)

**Notes on Response variable:**
    
When I use all 8 dummy columns for response, I get "bad shape" error below. I will try using just one Response columns(1 through 8) 
and try OnevsRest or some method...

- Test how separable the resposnes are by plotting a 2D projection of the feature space and coloring by response.
- For each type of response, find the most correlated feature.

In [ ]:
# Assign predictors to X and response to Y
X = df_predictors_selected
#y = df_response_selected
y = df_response_selected

In [ ]:
print X.shape
print y.shape

In [ ]:
# Check how separable the responses are using MDS
data_to_cluster = X
mds = MDS(n_components=2,random_state=1)
mds_transformed_data = mds.fit_transform(data_to_cluster)
mds_transformed_data_df = pd.DataFrame(mds_transformed_data, columns= ['MDS1','MDS2'])
mds_transformed_data_df.head()
colors = y
plt.scatter(mds_transformed_data_df['MDS1'].values,mds_transformed_data_df['MDS2'].values,c=colors)
plt.show()

In [ ]:

# Make 11 clusters using KMeans clustering
k = 8
km = KMeans(k)
cluster_info = km.fit(X)

# PCA to visulaize clusters
pca = PCA(n_components=2,copy=True)
transformed_data = pca.fit_transform(data_to_cluster)
transformed_data_df = pd.DataFrame(transformed_data, columns= ['PCA1','PCA2'])
transformed_data_df.head()
colors = cluster_info.labels_
plt.scatter(transformed_data_df['PCA1'].values,transformed_data_df['PCA2'].values,c=colors)
plt.show()


In [ ]:
# Check how separable the responses using PCA
data_to_cluster = X
pca = PCA(n_components=2,copy=True)
pca_transformed_data = pca.fit_transform(data_to_cluster)
pca_transformed_data_df = pd.DataFrame(pca_transformed_data, columns= ['PCA1','PCA2'])
pca_transformed_data_df.head()
colors = y
plt.scatter(pca_transformed_data_df['PCA1'].values,pca_transformed_data_df['PCA2'].values,c=colors)
plt.show()

In [ ]:
# # Compute pairwise correlations of all predictors
# df_predictors = df_expanded
# df_predictor_correlations = df_predictors.corr()


In [ ]:
# # Describe pairwise correlations
# _corr_threshold = 0.5
all_predictors = df_predictors_selected.columns
# for pred in all_predictors:
#     df_temp = df_predictor_correlations[pred]
#     max_corr = df_temp[[idx for idx in df_temp.index if pred not in idx]].max()
#     min_corr = df_temp.min()
#     if np.isnan(max_corr)==False:
#         if abs(min_corr)>max_corr:
#             other_pred = df_temp[df_temp==min_corr].index[0]
#             mcorr = min_corr
#         else:
#             other_pred = df_temp[df_temp==max_corr].index[0]
#             mcorr = max_corr
#         if abs(mcorr)>_corr_threshold:
#             print '** ',
#         print 'mcorr({})={:.4f} with {}'.format(pred, mcorr, other_pred)

In [ ]:
df_response

In [ ]:
for response in df_response[df_response.Response_1:
   
        score = {}
        score[response] = 
        score.append(df_response[response].corr(df_predictors_selected[predictor]))
        #print 'corr of response {} with feature {}: {}'.format(response,predictor,(df_response[response].corr(df_predictors_selected[predictor]
                                                                                                            # )))
    

In [ ]:
score_result_1 = {}
for predictor in X:    
    score_result_1[predictor]= X[predictor].corr(df_response['Response_1'])
    
inverse = [(value,key) for key, value in score_result_1.items() ]
print "Max correlated feature with Result_1: {}".format(max(inverse)[1])
print min(inverse)[1]

In [ ]:
from pylab import*
for ii in df_response:
    score_result_ii = {}
    for predictor in X:
        score_result_ii[predictor] = X[predictor].corr(df_response[ii])
    inverse = [(value,key) for key, value in score_result_ii.items()]
#     pos = arange(1)
#     plt.barh(pos,max(inverse)[0],align='center')
#     yticks(pos, (max(inverse)[1]))
#     plt.show()
    print "Max positively correlated feature with Result_{}: {}, correlation: {}".format(ii,max(inverse)[1],max(inverse)[0])
    print "Min negatively correlated feature with Result_{}: {},correlation: {}".format(ii,min(inverse)[1],min(inverse)[0])

In [ ]:
for 

# Learning curve

The point is to see if we have enough data. We will do this by determining cross-validated training and test scores for different training set sizes. I'm going to give this a shot with Logistic Regression using all features.

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [ ]:
# estimator = LogisticRegression()
# title = "Learning curve Logistic Regression"
# #cv = cross_validation.ShuffleSplit(digits.data.shape[0], n_iter=100,test_size=0.2, random_state=0)
# cv=StratifiedKFold(y,n_folds=5,shuffle=True,random_state=1)
# # plot_learning_curve(estimator, title, X, y, ylim=(0.7, 1.01), cv=cv, n_jobs=4)
# # plt.show()

# Model building:

**Make another version of the data where the minority response values are oversampled OR synthetically generated using SMOTE. First use the original dataset and then the altered for each model:[X,y OR X1,y1]**

1) Fit following models using all features:
    - Logistic Regression
    - Naive Bayes
    - K-Nearest Neighbor



2) Plot learning curve for any (?) model that gives a decent accuracy to see if we have enough data in hand!!



3) Fit ensemble models using all features:
    - RandomForest
    - Gradient Boosting Classifier
    - Voting Classifier (combo of models from Step#1: show that base classifiers are better than a coin toss)
    
 
**Other points to think about:**

1) K-fold cross-validation can be folded into the model: each run will give a score: we can get the median score.


2) Feature selection: Regularization (C which is 1/lambda.. C=0.1 means high penalty. Apply Lasso). Find coefficients and discuss the importance. Look for intuitive explanations. 
2.1) For Logistic Regression and Naive Bayes, we will get probabilities so besides just looking at the median CV score,will it be useful to look at the confusion matrix, find TPR, FPR, plot a ROC curve and determine AUC? Is that of interest to Prudential??


3) To simplify the model, if scores aren't great, we can even remove features showing multicollinearity. (Should we be looking at covariance?). Besides plotting a correlation matrix, will PCA help in finding out which features are colinear (since they will be on the same axis)?




In [ ]:
# Fit LOGISTIC REGRESSION model and compute the max cross_validation score over various K-folds.
model_lr = LogisticRegression(penalty = 'l1',C=0.05) # Instantiate Logistic Regression model
scores =[]
jj = 0
for ii in range(2,11):
    scores.append(np.median(cross_val_score(model_lr, X, y, cv=StratifiedKFold(y, ii, shuffle=False))))
    print "{}-fold cross-validation score: {}".format(ii,scores[jj])
    jj += 1
print '\n'"Best CV score: {}".format(max(scores))
with plt.style.context('fivethirtyeight'):
    plt.plot(range(2,11),scores)
    plt.xlabel('K-fold')
    plt.ylabel('Cross-val score')
plt.show()

In [ ]:
# find the CV score for 5-fold cross-validation
cv_score_5fold = cross_val_score(model_lr,X,y,cv=5).mean()
cv_score_5fold

In [ ]:
# To obtain coefficients of features, we need to do a test train split, fit, predict, score and get coefs
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
lr_fitted = model_lr.fit(X_train,y_train)
lr_fitted.score(X_test,y_test)
(lr_fitted.coef_).max()

In [ ]:
# Run KNN classifer
myknn = KNeighborsClassifier(5).fit(X_train,y_train)

# Output KNN score by running the model trained on training data on the test data
myknn.score(X_test, y_test)

In [ ]:
# Run 5-fold cross-validation
model = KNeighborsClassifier(5)
data = X
label = y
cross_val_score(model, data, label, cv=StratifiedKFold(label, 5, shuffle=False))

In [ ]:
# Fit a Naive Bayes Model
nb = MultinomialNB()
nb.fit(X_train, y_train)
preds = nb.predict(X_test)
print metrics.accuracy_score(y_test, preds)
print metrics.confusion_matrix(y_test, preds)

**Conclusions:**

Using all features of df_sampled, all three models are performing poorly.
- LR_accuracy: 35%
- KNN_accruacy: 21%
- Naive Bayes accuracy: 19%


**Next steps:**

- No point doing a Voting Classifier with above methods since individually all of them are bad.

- Fit gradient boosting classifier and do a grid search

- Synthetically upsample (SMOTE) response # 3,4 and downsample response 6,8 and see if performance of gradient boosting improves

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.grid_search import GridSearchCV

X = df_predictors_selected
y = df_response_selected

CV_FOLDS = 2

# Set up grid search
pgrid = {'learning_rate': [0.05, 0.1, 0.5], #'n_estimators': [1, 10, 100, 1000], 
         'subsample': [0.6, 0.75, 0.9],
         'max_depth': [3,7,15]
        }

clf_grdbst = GradientBoostingClassifier(verbose=True)

grid = GridSearchCV(clf_grdbst, param_grid=pgrid, cv = CV_FOLDS)
grid.fit(X, y)

In [ ]:
grid.best_estimator_.score(X,y)

In [ ]:
grid.best_params_

In [ ]:
grid.best_score_

In [ ]:
grid.best_estimator_

**Gradient boosting classifier accuracy = 46.6%**